In [ ]:
"""

chime data is 4gb

downloaded from 

"""

In [111]:
import glob
import pandas as pd
import numpy as np
import re
import json
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression




In [6]:
# constants
data_folder = 'data/chime_home/'
data_chunks = f'{data_folder}chunks/'

In [5]:
eval_dev_chunks = glob.glob('data/chime_home/*.csv')
raw_chunks = glob.glob('data/chime_home/chunks/*.csv')

files = {}
for filename in eval_dev_chunks:
    files[filename.split('/')[-1]] = pd.read_csv(filename, header=None, names=['index', 'filename'])
    
dev_ref_df = files['development_chunks_refined.csv']
dev_ref_df = dev_ref_df.drop(columns=['index'])

In [58]:
results = []
for filename in dev_ref_df['filename']:
    chunk_df = pd.read_csv(f'{data_chunks}{filename}.csv', header=None)
    chunk_t = chunk_df.set_index(0).T
    results.append(chunk_t)
    
concat_df = pd.concat(results)

In [66]:
human_voices_re = re.compile(r'[cmf]') # child male female
concat_df['human_voice'] = concat_df['majorityvote'].apply(lambda x: 1 if human_voices_re.search(x) else 0)

SyntaxError: invalid syntax (<ipython-input-66-012550b79a61>, line 1)

In [90]:
concat_df['human_voice'].value_counts()

1    1314
0     632
Name: human_voice, dtype: int64

In [7]:
dev_refined_concat_filename = 'development_chunks_refined_concat_df.pkl'


In [148]:
concat_df.to_pickle(f'{dev_refined_concat_filename}')

In [9]:

with open(f'{dev_refined_concat_filename}', 'rb') as readfile:
    df = pickle.load(readfile)


In [37]:
# Feature extraction example
import librosa

# Load the example clip
def extract_features(filename):
    y, sr = librosa.load(f'{filename}')

    # Set the hop length; at 22050 Hz, 512 samples ~= 23ms
    hop_length = 512

    # Separate harmonics and percussives into two waveforms
    y_harmonic, y_percussive = librosa.effects.hpss(y)

    # Beat track on the percussive signal
    tempo, beat_frames = librosa.beat.beat_track(y=y_percussive,
                                                 sr=sr)

    # Compute MFCC features from the raw signal
    mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)

    # And the first-order differences (delta features)
    mfcc_delta = librosa.feature.delta(mfcc)

    # Stack and synchronize between beat events
    # This time, we'll use the mean value (default) instead of median
    beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]),
                                        beat_frames)

    # Compute chroma features from the harmonic signal
    chromagram = librosa.feature.chroma_cqt(y=y_harmonic,
                                            sr=sr)

    # Aggregate chroma features between beat events
    # We'll use the median value of each feature between beat frames
    beat_chroma = librosa.util.sync(chromagram,
                                    beat_frames,
                                    aggregate=np.median)

    # Finally, stack all beat-synchronous features together
    beat_features = np.vstack([beat_chroma, beat_mfcc_delta])
    
    return {
        'mfcc': mfcc,
        'mfcc_delta': mfcc_delta,
        'beat_mfcc_delta': beat_mfcc_delta,
        'chromagram': chromagram,
        'beat_chroma': beat_chroma,
        'beat_features': beat_features
    }

In [ ]:
def extract_features_df(df, column_name = 'chunkname'):
    results = {}
    total = df.shape[0]
    count = 0
    for index, row in df.iterrows():
        print('what is left', (count/total) * 100, '%')
        chunkname = row[column_name]
        chunk_filename = f'{data_chunks}{chunkname}{codec}'
        features = extract_features(chunk_filename)
        results[chunkname] = features
        
        count += 1
        
    return results


In [ ]:
all_features = extract_features_df(df)

In [11]:
refined_features = 'development_chunks_refined_features.pkl'

In [141]:

with open(f'{refined_features}', 'wb') as outfile:
    pickle.dump(all_features, outfile)
#     json.dump(all_features, outfile, indent=4)

In [12]:
with open(f'{refined_features}', 'rb') as readfile:
    all_features = pickle.load(readfile)


In [15]:
len(all_features.keys())

1946

dict_keys(['CR_lounge_220110_0731.s0_chunk27', 'CR_lounge_220110_0731.s0_chunk18', 'CR_lounge_220110_0731.s0_chunk70', 'CR_lounge_220110_0731.s0_chunk0', 'CR_lounge_220110_0731.s0_chunk39', 'CR_lounge_220110_0731.s0_chunk5', 'CR_lounge_220110_0731.s0_chunk24', 'CR_lounge_220110_0731.s0_chunk72', 'CR_lounge_220110_0731.s0_chunk15', 'CR_lounge_220110_0731.s0_chunk16', 'CR_lounge_220110_0731.s0_chunk35', 'CR_lounge_220110_0731.s0_chunk69', 'CR_lounge_220110_0731.s0_chunk20', 'CR_lounge_220110_0731.s0_chunk74', 'CR_lounge_220110_0731.s0_chunk26', 'CR_lounge_220110_0731.s0_chunk46', 'CR_lounge_220110_0731.s0_chunk11', 'CR_lounge_220110_0731.s0_chunk19', 'CR_lounge_220110_0731.s0_chunk71', 'CR_lounge_220110_0731.s0_chunk22', 'CR_lounge_220110_0731.s0_chunk7', 'CR_lounge_220110_0731.s0_chunk12', 'CR_lounge_220110_0731.s0_chunk40', 'CR_lounge_220110_0731.s0_chunk21', 'CR_lounge_220110_0731.s0_chunk41', 'CR_lounge_220110_0731.s0_chunk30', 'CR_lounge_220110_0731.s0_chunk17', 'CR_lounge_220110_07

In [38]:

codec ='.48kHz.wav'
chunkname = 'CR_lounge_220110_0731.s0_chunk27'
chunk_filename = f'{data_chunks}{chunkname}{codec}'
features_one = extract_features(chunk_filename)


In [39]:
len(features_one['mfcc_delta'])

13

In [125]:
len(features_one['beat_mfcc_delta'])

26

In [62]:
scaler = StandardScaler()

variance_explained = 0.95
n_components = 10
# pca = PCA(variance_explained)
pca = PCA(n_components = n_components)

features_one_df = pd.DataFrame(features_one['mfcc'])
# features_one_train = scaler.fit_transform(features_one_df)
principalComponents = pca.fit_transform(features_one_train)
# principalDf = pd.DataFrame(data = principalComponents
#              , columns = ['principal component 1', 'principal component 2'])


In [68]:
pcaDF = pd.DataFrame(data=principalComponents)

In [70]:
pcaDF.shape

(13, 10)

In [63]:
principalComponents.explained_variance_ratio_

AttributeError: 'numpy.ndarray' object has no attribute 'explained_variance_ratio_'

In [71]:
df.head(1)

,segmentname,chunknumber,framestart,annotation_a1,session_a1,annotation_a2,session_a2,annotation_a3,session_a3,majorityvote,chunkname,human_voice
1,CR_lounge_220110_0731.s0,27,5184000,cp,4.0,cop,2.0,cop,4.0,cop,CR_lounge_220110_0731.s0_chunk27,1


In [104]:
def create_overwhelming(df, features):
    results = []
    
    for index, row in df.iterrows():
        chunkname = row['chunkname']
        feature = features[chunkname]['mfcc']
        is_human = row['human_voice']
        flattened = np.array(feature).flatten().tolist()
        flattened.insert(0, chunkname)
        flattened.insert(0, is_human)
        results.append(flattened)
                
    return pd.DataFrame(results)
        

In [98]:
edf = pd.DataFrame([['a', 1, 2], ['b', 2, 3]], columns=['chunkname', 'human_voice', 'two'])


In [101]:
create_overwhelming(edf, {'a': [[1,2,3], [4, 5, 6]], 'b': [[2,4, 5], [3, 6, 9]]})

,0,1,2,3,4,5,6,7
0,1,a,1,2,3,4,5,6
1,2,b,2,4,5,3,6,9


In [105]:
mdf = create_overwhelming(df, all_features)

In [119]:
mdf

,0,1,2,3,4,5,6,7,8,9,...,2241,2242,2243,2244,2245,2246,2247,2248,2249,2250
0,1,CR_lounge_220110_0731.s0_chunk27,-460.032135,-450.238831,-433.465759,-407.766632,-386.450592,-373.867249,-373.253204,-376.718170,...,-6.015262,-10.757653,-11.165491,-10.607825,-13.730804,-6.789348,-6.029935,-3.624697,-0.950168,-2.479859
1,1,CR_lounge_220110_0731.s0_chunk18,-502.901642,-439.566284,-310.146912,-196.677322,-143.958191,-121.890251,-115.686531,-125.213715,...,3.277682,-0.587493,0.046199,-4.752885,-4.910209,-0.153703,-1.368832,-1.928573,-0.542693,1.211689
2,1,CR_lounge_220110_0731.s0_chunk70,-480.194031,-484.733093,-501.425995,-518.882568,-535.860840,-542.206970,-546.596130,-554.124207,...,2.304256,-0.185979,-5.010571,-1.363904,-4.386158,-3.691793,-0.323971,0.286205,-1.644120,-5.086028
3,0,CR_lounge_220110_0731.s0_chunk0,-529.249451,-530.759338,-543.135010,-550.916077,-557.480408,-557.828857,-552.620728,-554.126343,...,5.934577,7.152072,11.320415,5.753222,1.330544,0.047888,0.675164,3.985723,1.067848,2.220659
4,1,CR_lounge_220110_0731.s0_chunk39,-298.271423,-308.898071,-320.336212,-311.373444,-325.856323,-309.311768,-279.476776,-276.182343,...,0.901431,1.238960,1.377512,2.657545,5.603203,5.593097,3.590163,-0.245241,1.755665,1.943853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1941,1,CR_lounge_270110_1632.s3900_chunk10,-354.801300,-357.910431,-368.541931,-373.195923,-373.310944,-382.509979,-384.369232,-388.606354,...,4.950514,10.983016,10.041251,7.969370,11.732731,11.466062,8.036340,4.825058,8.576981,16.337179
1942,1,CR_lounge_270110_1632.s3900_chunk65,-491.656494,-489.575439,-493.648773,-500.193115,-511.542725,-508.556885,-505.326843,-491.686676,...,4.541065,8.230209,3.416269,-4.567317,-7.928992,-2.784325,-1.224775,3.923793,8.971301,10.247086
1943,1,CR_lounge_270110_1632.s3900_chunk30,-149.878922,-174.841553,-231.188690,-234.239349,-237.411102,-248.221985,-254.781311,-247.807007,...,8.815567,6.748887,1.594312,-2.088048,-2.859728,-3.648098,-3.201086,-9.748062,-12.836238,-15.898949
1944,1,CR_lounge_270110_1632.s3900_chunk4,-264.232422,-222.985672,-199.127640,-203.703552,-231.866257,-250.831497,-242.183167,-268.898834,...,6.745681,5.941745,4.545578,4.087250,5.432576,6.580695,6.017241,5.096549,5.172022,6.060733


In [107]:
from sklearn.model_selection import train_test_split
X = mdf.drop(columns=[0, 1], axis=1)
y = mdf[0]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=None
)

In [128]:
sum(y) / len(y)

0.6752312435765673

In [112]:
logRegM = LogisticRegression()
logRegM.fit(X_train, y_train)

/Users/greg/anaconda3/envs/metis/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

array([1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,